# Scripts and Drafts for Arrow-Debreu Market

In [2]:
using Pkg
Pkg.activate("../")
using Revise
using Random, SparseArrays, LinearAlgebra
using JuMP, MosekTools
using Plots, LaTeXStrings, Printf
import MathOptInterface as MOI

using ExchangeMarket
using DRSOM

include("../tools.jl")
include("../plots.jl")
switch_to_pdf(; bool_use_html=true)

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`
┌ Warning: `PlotlyKaleido` 2.3.0 is not compatible with this version of `Plots`. The declared compatibility is 1.
└ @ Plots /Users/brent/.julia/packages/Plots/FFuQi/src/backends.jl:55


:html

## Test run: when is it hard?

In [3]:
include("./func.jl")

Random.seed!(1)
m = 3
n = 3
ρ = -20
f0 = generate_triagular_market(m, n; ρ=ρ, scale=1.0)

linconstr = LinearConstr(1, n, ones(1, n), [1.0])
# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
alg = HessianBar(
    n, m, p₀;
    linconstr=linconstr,
)
alg.linsys = :direct

Cardinal Optimizer v7.2.4. Build date Dec  6 2024
Copyright Cardinal Operations 2024. All Rights Reserved
ArrowDebreuMarket initialization started...
ArrowDebreuMarket initialized in 0.011938095092773438 seconds
ArrowDebreuMarket initialization started...
ArrowDebreuMarket initialized in 0.031944990158081055 seconds


:direct

In [4]:
alg.p = [0.5, 0.0, 0.5]

3-element Vector{Float64}:
 0.5
 0.0
 0.5

In [5]:
irange = [-6.7:0.00005:-6.55...]
zrange = []
for i in irange
    alg.p[2:end-1] .= 10^i
    update_budget!(f1, alg.p)
    play!(alg, f1)
    grad!(alg, f1)
    eval!(alg, f1)
    push!(zrange, abs(alg.∇[2]))
    # push!(zrange, alg.∇[2])
end

In [8]:
fig = generate_empty(; shape=:square, settick=false)
plot!(
    fig, 10.0 .^ irange, zrange, label="", linewidth=4,
    marker=nothing,
    markershape=:circle, markerstrokecolor=nothing, markerstrokewidth=0.0,
    xscale=:log10, yscale=:log10,
    ylabel=L"|z^{(2)}(\mathbf p)|",
    xlabel=L"p^{(2)}",
    labelfontsize=20,
    xtickfont=font(20),
    ytickfont=font(20),
    legendfontsize=20,
    titlefontsize=20,
    size=(600, 320),
    # tex_output_standalone=true
)
savefig(fig, "/tmp/arrow_near_leontief_$(n).pdf")
savefig(fig, "/tmp/arrow_near_leontief_$(n).tex")
irange[argmin(zrange)]

-6.62265